In [ ]:
# imports

# external modules
import sys
import os
import numpy as np
import pandas as pd
import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
# framework modules
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import training.prepare_training_set
importlib.reload(training.prepare_training_set)
from training.prepare_training_set import prepare_training_data_from_files
import training.patternfiltering
importlib.reload(training.patternfiltering)
from training.patternfiltering import contains_any_pattern
import datagen.fake_anomaly
importlib.reload(datagen.fake_anomaly)

In [ ]:
# load the evaluation set

me = 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1'

files = ([
    '../data/data/ZeroBias-Run2023C-PromptReco-v1-DQMIO-{}_preprocessed.parquet'.format(me)
])
kwargs = ({
    'verbose': True,
    'entries_threshold': 10000,
    'skip_first_lumisections': 5,
    'veto_patterns': [np.zeros((2,2)), np.zeros((2,1)), np.zeros((1,2))] # stricter than training set
    #'veto_patterns': [np.zeros((2,2)), np.zeros((3,1)), np.zeros((1,3))] # same as training set
})
(eval_data, eval_runs, eval_lumis) = prepare_training_data_from_files(files, **kwargs)

In [ ]:
# make a mask where values are often zero

shape_mask = (np.sum(eval_data[:,:,:,0]==0, axis=0)>len(eval_data)*0.5)

# temp: also set edges to zero
shape_mask[0,:] = 1
shape_mask[-1,:] = 1
shape_mask[:,0] = 1
shape_mask[:,-1] = 1

fig,ax = plt.subplots()
plot_histogram(shape_mask, fig=fig, ax=ax, caxrange=(-0.01,1))
ax.text(0.02, 1.02, 'Shape mask', transform=ax.transAxes, fontsize=12)

In [ ]:
# load keras model

modelbase = '../models/output_20231123/model_20231123_Run2023C-v1'
modelname = '{}_{}.keras'.format(modelbase, me)
model = keras.models.load_model(modelname)

In [ ]:
# load average occupancy or error of training set

avgresponsename = '{}_{}_avgoccupancy.npy'.format(modelbase, me)
avgresponse = np.load(avgresponsename)
avgresponse = np.square(avgresponse)

fig,ax = plt.subplots()
plot_histogram(avgresponse, fig=fig, ax=ax)
ax.text(0.02, 1.02, 'Average response on training set', transform=ax.transAxes, fontsize=12)
avgresponse[avgresponse==0] = 1
avgresponse = np.expand_dims(avgresponse, axis=2)

In [ ]:
# make a collection of fake anomalies

nanomalies = 6000
ntimesteps = 3
random_indices = np.random.choice(len(eval_data), size=nanomalies, replace=True)
random_indices[random_indices<ntimesteps-1] = ntimesteps-1

anomalies = np.zeros((nanomalies*ntimesteps, eval_data.shape[1], eval_data.shape[2], 1))
params = []
for newidx,origidx in enumerate(random_indices):
    hist = eval_data[origidx-ntimesteps+1:origidx+1,:,:,0]
    (anomalous_hist, paramdict) = datagen.fake_anomaly.dead_rectangle(hist, rectangle_min_pixels=2, shape_mask=~shape_mask)
    #(anomalous_hist, paramdict) = datagen.fake_anomaly.dead_sector(hist)
    #(anomalous_hist, paramdict) = datagen.fake_anomaly.hot_rectangle(hist, shape_mask=~shape_mask)
    #(anomalous_hist, paramdict) = datagen.fake_anomaly.hot_sector(hist)
    anomalies[ntimesteps*newidx:ntimesteps*(newidx+1),:,:,0] = anomalous_hist
    params.append(paramdict)
print('Shape of anomalies array: {}'.format(anomalies.shape))

In [ ]:
# evaluate the model

predictions = model.predict(eval_data)
predictions[predictions<0] = 0.
predictions[:,shape_mask] = 0.

predictions_anomalies = model.predict(anomalies)
predictions_anomalies[predictions_anomalies<0] = 0.
predictions_anomalies[:,shape_mask] = 0.

In [ ]:
# calculate squared difference
errors = np.square(eval_data - predictions)
errors[:,shape_mask] = 0.
errors_anomalies = np.square(anomalies - predictions_anomalies)
errors_anomalies[:,shape_mask] = 0.

# space correction
errors_space_corrected = errors/avgresponse
errors_anomalies_space_corrected = errors_anomalies/avgresponse

# extra space correction: divide by average occupancy of each lumisection
occupancy_perls = np.square(np.sum(eval_data, axis=(1,2))[:,0])
mean_occupancy_perls = np.mean(occupancy_perls)
occupancy_perls /= mean_occupancy_perls
errors_space_corrected = np.divide(errors_space_corrected, occupancy_perls[:,None,None,None])
occupancy_anomalies_perls = np.square(np.sum(anomalies, axis=(1,2))[:,0])
occupancy_anomalies_perls /= mean_occupancy_perls # use mean from non-anomalous data here for consistency
errors_anomalies_space_corrected = np.divide(errors_anomalies_space_corrected, occupancy_anomalies_perls[:,None,None,None])

# time correction
errors_time_corrected = np.zeros(errors_space_corrected.shape)
for i in range(ntimesteps-1, len(errors_space_corrected)):
    errors_time_corrected[i] = np.prod(errors_space_corrected[i-ntimesteps+1:i+1], axis=0)
errors_anomalies_time_corrected = np.zeros((nanomalies, eval_data.shape[1], eval_data.shape[2], eval_data.shape[3]))
for i in range(len(errors_anomalies_time_corrected)):
    errors_anomalies_time_corrected[i:i+1] = np.prod(errors_anomalies_space_corrected[ntimesteps*i:ntimesteps*(i+1)], axis=0)
    
# remove superfluous anomalies that were only introduced to be able to do the time correction
anomalies = anomalies[::ntimesteps]
predictions_anomalies = predictions_anomalies[::ntimesteps]
errors_anomalies = errors_anomalies[::ntimesteps]
errors_anomalies_space_corrected = errors_anomalies_space_corrected[::ntimesteps]
print(anomalies.shape)
print(predictions_anomalies.shape)
print(errors_anomalies.shape)
print(errors_anomalies_space_corrected.shape)
print(errors_anomalies_time_corrected.shape)

In [ ]:
# make plots of evaluation on supposedly non-anomalous histograms

nplots = 5
plotids = np.random.choice(len(eval_data), size=nplots, replace=False)

for i in plotids:
    fig,axs = plt.subplots(figsize=(30,6), ncols=5)
    plot_histogram(eval_data[i,:,:,0], fig=fig, ax=axs[0])
    plot_histogram(predictions[i,:,:,0], fig=fig, ax=axs[1])
    plot_histogram(errors[i,:,:,0], fig=fig, ax=axs[2], caxrange=(-0.001, 0.01))
    plot_histogram(errors_space_corrected[i,:,:,0], fig=fig, ax=axs[3], caxrange=(-0.01, 0.5))
    plot_histogram(errors_time_corrected[i,:,:,0], fig=fig, ax=axs[4], caxrange=(-0.01, 0.5))
    axs[0].text(0.02, 1.02, 'Run {}, LS {}'.format(eval_runs[i], eval_lumis[i]), transform=axs[0].transAxes, fontsize=12)

In [ ]:
# make plots of evaluation on anomalous histograms

nplots = 5
plotids = np.random.choice(len(anomalies), size=nplots, replace=False)

for i in plotids:
    fig,axs = plt.subplots(figsize=(30,6), ncols=5)
    plot_histogram(anomalies[i,:,:,0], fig=fig, ax=axs[0])
    plot_histogram(predictions_anomalies[i,:,:,0], fig=fig, ax=axs[1])
    plot_histogram(errors_anomalies[i,:,:,0], fig=fig, ax=axs[2], caxrange=(-0.001, 0.01))
    plot_histogram(errors_anomalies_space_corrected[i,:,:,0], fig=fig, ax=axs[3], caxrange=(-0.01, 0.5))
    plot_histogram(errors_anomalies_time_corrected[i,:,:,0], fig=fig, ax=axs[4], caxrange=(-0.01, 0.5))
    axs[0].text(0.02, 1.02, 'Fake anomaly', transform=axs[0].transAxes, fontsize=12)

In [ ]:
# scan thresholds to make a ROC curve

#thresholds = [0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 1, 1.25, 1.5]
thresholds = np.logspace(-4 ,0, num=30)
patterns = [-np.ones((2,2)), -np.ones((2,1)), -np.ones((1,2))]

s_eff = []
b_eff = []
for threshold in thresholds:
    errors_scaled = np.where(errors_time_corrected>threshold, -1, errors_time_corrected)
    errors_anomalies_scaled = np.where(errors_anomalies_time_corrected>threshold, -1, errors_anomalies_time_corrected)
    flags = contains_any_pattern(errors_scaled[:,:,:,0], patterns)
    flags_anomalies = contains_any_pattern(errors_anomalies_scaled[:,:,:,0], patterns)
    print('Threshold: {}'.format(threshold))
    print('{} out of {} non-anomalous histograms were tagged'.format(sum(flags), len(flags)))
    print('{} out of {} anomalous histograms were tagged'.format(sum(flags_anomalies), len(flags_anomalies)))
    s_eff.append( sum(flags_anomalies)/len(flags_anomalies) )
    b_eff.append( sum(flags)/len(flags) )

fig,ax = plt.subplots()
ax.scatter(b_eff, s_eff, c='b', s=10)
ax.set_xlabel('False alarm rate', fontsize=15)
ax.set_ylabel('True anomaly efficiency', fontsize=15)
ax.grid()
#ax.set_ylim((0,1))
ax.set_xscale('log')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.scatter(b_eff, s_eff, c='b', s=10)
ax.set_xlabel('False alarm rate', fontsize=15)
ax.set_ylabel('True anomaly efficiency', fontsize=15)
ax.grid()
#ax.set_ylim((0,1))
ax.set_xscale('log')
plt.show()

In [ ]:
# for a given threshold, investigate mistagged good histograms

nplots = 5
threshold = 0.75
errors_scaled = np.where(errors_time_corrected>threshold, -1, errors_time_corrected)
flags = contains_any_pattern(errors_scaled[:,:,:,0], patterns)
print('Found {} candidates, of which plotting {}'.format(sum(flags), nplots))
plotids = np.random.choice(np.arange(len(eval_data))[flags], size=nplots, replace=False)

for i in plotids:
    fig,axs = plt.subplots(figsize=(30,6), ncols=5)
    plot_histogram(eval_data[i,:,:,0], fig=fig, ax=axs[0])
    plot_histogram(predictions[i,:,:,0], fig=fig, ax=axs[1])
    plot_histogram(errors[i,:,:,0], fig=fig, ax=axs[2], caxrange=(-0.001, 0.01))
    plot_histogram(errors_space_corrected[i,:,:,0], fig=fig, ax=axs[3], caxrange=(-0.01, threshold))
    plot_histogram(errors_time_corrected[i,:,:,0], fig=fig, ax=axs[4], caxrange=(-0.01, threshold))
    axs[0].text(0.02, 1.02, 'Run {}, LS {}'.format(eval_runs[i], eval_lumis[i]), transform=axs[0].transAxes, fontsize=12)
    plt.show()

In [ ]:
# for a given threshold, investigate missed anomalous histograms

nplots = 5
threshold = 0.75
errors_anomalies_scaled = np.where(errors_anomalies_time_corrected>threshold, -1, errors_anomalies_time_corrected)
flags_anomalies = contains_any_pattern(errors_anomalies_scaled[:,:,:,0], patterns)
print('Found {} candidates, of which plotting {}'.format(sum(~flags_anomalies), nplots))
plotids = np.random.choice(np.arange(len(anomalies))[~flags_anomalies], size=nplots, replace=False)

for i in plotids:
    fig,axs = plt.subplots(figsize=(30,6), ncols=5)
    plot_histogram(anomalies[i,:,:,0], fig=fig, ax=axs[0])
    plot_histogram(predictions_anomalies[i,:,:,0], fig=fig, ax=axs[1])
    plot_histogram(errors_anomalies[i,:,:,0], fig=fig, ax=axs[2], caxrange=(-0.001, 0.01))
    plot_histogram(errors_anomalies_space_corrected[i,:,:,0], fig=fig, ax=axs[3], caxrange=(-0.01, threshold))
    plot_histogram(errors_anomalies_time_corrected[i,:,:,0], fig=fig, ax=axs[4], caxrange=(-0.01, threshold))
    axs[0].text(0.02, 1.02, 'Fake anomaly', transform=axs[0].transAxes, fontsize=12)